In [40]:
from matplotlib import pyplot as plt
from matplotlib.font_manager import FontProperties
from math import log
import pickle
import operator
import numpy as np

In [41]:
def creatDataSet():
    dataset = [[0,0,0,0,'no'],
               [0,0,0,1,'no'],
               [0,1,0,1,'yes'],
               [0,1,1,0,'yes'],
               [0,0,0,0,'no'],
               [1,0,0,0,'no'],
               [1,0,0,1,'no'],
               [1,1,1,1,'yes'],
               [1,0,1,2,'yes'],
               [2,0,1,2,'yes'],
               [2,0,1,1,'yes'],
               [2,1,0,1,'yes'],
               [2,1,0,2,'yes'],
               [2,0,0,0,'no']]
    feature_names = ['Age','Work','Home','Loan']
    return dataset, feature_names

In [92]:
def createTree(dataset, feature_names, best_feature_order):
    """
    创建决策树
    feature_names: dataset每一列的特征名
    feature_order：决策树的特征顺序（优先级高的在前面），为列表
    返回值：决策树字典
    """
    labels = [each[-1] for each in dataset]
    
    # 递归结束条件 -- 1.标签只有一个类别 2.已经使用完所有的特征
    if labels.count(labels[0]) == len(labels):
        return labels[0]
    if len(dataset[0]) == 1:
        return majorityCnt(labels)  # 所有特征都用完，此分支的类别由多数标签决定
    
    # 递归
    # 确定当前最佳标签
    best_feature_index = choosBestFeature(dataset)
    
    #print(best_feature_index)
    #print(len(dataset[0]))
    
    best_feature_name = feature_names[best_feature_index]
    best_feature_order.append(best_feature_name)
    del feature_names[best_feature_index]
    
    # 创建树根
    my_tree = {best_feature_name:{}}
    
    # 对最佳标签进行操作
    feature_values = [each[best_feature_index] for each in dataset]
    unique_feature_values = set(feature_values)
    
    for value in unique_feature_values:
        sub_feature_names = feature_names[:]
        my_tree[best_feature_name][value] = createTree(splitDataSet(dataset,best_feature_index,value), sub_feature_names, best_feature_order)
    
    return my_tree

    
def majorityCnt(labels):
    """
    返回标签名
    输入：列表
    输出：字符串
    """
    labels_count = {}
    for label in labels:
        if label not in labels_count:
            labels_count[label] = 0
        else:
            labels_count[label] += 1
    return sorted(labels_count.items(), 
                  key=operator.itemgetter(1),reverse=True)[0][0]


def choosBestFeature(dataset):
    """
    返回值：index for feature
    """
    base_entropy = calcEntropy(dataset)
    best_gain = 0
    best_feature_index = None
    
    n_features = len(dataset[0]) - 1
    #print(f"n_features:{n_features}")
    #print(f"base_entropy:{base_entropy}")
    #print(dataset)
    
    for feature_index in range(n_features):
        
        feature_values = [each[feature_index] for each in dataset]
        uniqe_feature_values = set(feature_values)
        
        new_entropy = 0
        for uniqe_feature_value in uniqe_feature_values:
            sub_dataset = splitDataSet(dataset,feature_index,uniqe_feature_value)
            single_entropy = calcEntropy(sub_dataset)
            prop = len(sub_dataset) / float(len(dataset))
            new_entropy += prop*single_entropy
        new_gain = base_entropy - new_entropy
        #print(best_gain, new_gain)
        if new_gain > best_gain:
            best_gain = new_gain
            best_feature_index = feature_index
        
    return best_feature_index
    

def calcEntropy(dataset):
    
    labels = [each[-1] for each in dataset]
    labels_count = {}
    
    for label in labels:
        if label not in labels_count:
            labels_count[label] = 1
        else:
            labels_count[label] += 1
    #print(labels_count)
    #print(labels_count)
    entropy = 0
    for label_name in labels_count:
        prop = labels_count[label_name] / float(len(labels))
        #print(prop)
        label_entropy = prop * log(prop,2) if prop != 0 else 0
        entropy -= label_entropy
    #print(entropy)
        
    return entropy


def splitDataSet(dataset, feature_index, value):
    new_dataset = []
    for line in dataset:
        if line[feature_index] == value:
            new_dataset.append(line[:feature_index] + line[feature_index+1:])
    #print(np.array(dataset).shape, np.array(new_dataset).shape)
    return new_dataset

In [94]:
dataset, feature_names = creatDataSet()
best_feature_order = []
tree = createTree(dataset, feature_names, best_feature_order)
tree

{'Work': {0: {'Home': {0: 'no', 1: 'yes'}}, 1: 'yes'}}

In [83]:
dict_ = {'a':2,'b':1,'c':3}
print(dict_.items())
sorted(dict_.items(), key=operator.itemgetter(1))
len(dict_)

dict_items([('a', 2), ('b', 1), ('c', 3)])


3

In [84]:
log(4,2)

2.0